In [1]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("/root/workspace/read")

In [2]:
from pyserini.search.lucene import LuceneSearcher

/opt/conda/envs/read/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from pyserini.search.lucene import LuceneSearcher
from pyserini.search.faiss import FaissSearcher, DprQueryEncoder
from pyserini.search.hybrid import HybridSearcher

ssearcher = LuceneSearcher.from_prebuilt_index("wikipedia-dpr")

encoder = DprQueryEncoder("facebook/dpr-question_encoder-multiset-base")
dsearcher = FaissSearcher.from_prebuilt_index(
    "wikipedia-dpr-multi-bf", encoder
)
searcher = HybridSearcher(dsearcher, ssearcher)

Attempting to initialize pre-built index wikipedia-dpr.


index-wikipedia-dpr-20210120-d1b9e6.tar.gz:  53%|█████████        | 4.57G/8.55G [11:01<09:37, 7.41MB/s]


KeyboardInterrupt: 

In [10]:
import pandas as pd

df = pd.read_json("../data/totto/augmented/train.jsonl", lines=True)

In [11]:
import json

def process_content(raw_content: str) -> str:
    title, content = json.loads(raw_content)["contents"].split("\n", 1)
    return title[1:-1], content

In [13]:
from read.utils.table import linearize
from torchmetrics import RetrievalHitRate, RetrievalRPrecision, RetrievalRecall, RetrievalPrecision, RetrievalMRR, RetrievalMAP


k2metrics = {k: [RetrievalHitRate(k=k), RetrievalRPrecision(k=k), RetrievalRecall(k=k), RetrievalPrecision(k=k), RetrievalMRR(k=k), RetrievalMAP(k=k)] for k in [1, 5, 10]}


preds = []
indices = []
golds = []

prediction_data = []

df = df.sample(frac=1)

for idx, row in df.iloc[:100].iterrows():
    one_prediction = []
    table = row["table"]
    query = linearize(table, row["highlighted_cells"], row_sep=" ; ", value_sep=" : ", includes_header=True, return_text=True)
    
    hits = searcher.search(query, k=10)
    
    for hit in hits:
        doc = ssearcher.doc(hit.docid)
        title, text = process_content(doc.raw())
        
        preds.append(hit.score)
        indices.append(idx)
        golds.append(title == row["table_page_title"])
        
        # if title == row["table_page_title"]:
        #     print(f"Title: '{title}' with query ({query}) for sentence ({row['sentence']})\n")
        #     print(f"Label: {row['negatives'] is not None}\n and index {idx}")
        #     print(f"Text: {text}\n")

        one_prediction.append({"idx": idx, "title": title, "text": text, "score": hit.score, "gold": row["table_page_title"], "table": table, "query": query, "sentence": row["sentence_annotations"]})
    prediction_data.append(one_prediction)

In [ ]:
import torch

result = {}

preds = torch.tensor(preds)
golds = torch.tensor(golds)
indices = torch.tensor(indices)

for k, metrics in k2metrics.items():
    for metric in metrics:
        result[f"{metric.__class__.__name__}@{k}"] = metric(preds, golds, indices)

In [ ]:
result

[autoreload of torch.overrides failed: Traceback (most recent call last):
  File "/nas/home/minhpham/miniconda3/envs/read/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/nas/home/minhpham/miniconda3/envs/read/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 455, in superreload
    module = reload(module)
  File "/nas/home/minhpham/miniconda3/envs/read/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 843, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/nas/home/minhpham/miniconda3/envs/read/lib/python3.8/site-packages/torch/overrides.py", line 1512, in <module>
    has_torch_function = _add_docstr(
RuntimeError: function '_has_torch_function' already has a docstring
]
[auto